<a href="https://colab.research.google.com/github/nepridumalnik/Jupyter-Notebooks/blob/master/pytorch/tutorials/PyTorch_Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Быстрый старт
## Работа с данными

PyTorch имеет два примитива для работы с данными - `torch.utils.data.DataLoader` и `torch.utils.data.Dataset`. `Dataset` хранит образцы и соответствующие им лейблы, а `DataLoader` оборачивают итераторы вокруг данных.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


PyTorch предоставляет домен-специфичные библиотеки, такие как [TorchText](https://pytorch.org/text/stable/index.html), [TorchVision](https://pytorch.org/vision/stable/index.html) и [TorchAudio](https://pytorch.org/audio/stable/index.html), каждый из которых содержит наборы данных.

Набор данных модуля `torchvision.datasets` содержит `Dataset` объекты наборов данных для многих реальных визуальных данных. Каждый `Dataset` из `TorchVision` содержит два аргумента `transform` и `target_transform` для модификации образцов и лейблов соответственно.

In [2]:
DATASET_ROOT: str = "data"

# Загрузка тренировочных данных открытого набора данных.
training_data = datasets.FashionMNIST(
    root=DATASET_ROOT,
    train=True,
    download=True,
    transform=ToTensor(),
)

# Загрузка тестовых данных открытого набора данных.
test_data = datasets.FashionMNIST(
    root=DATASET_ROOT,
    train=False,
    download=True,
    transform=ToTensor(),
)


Мы передаём `Dataset` как аргумент в `DataLoader`, это оборачивает в итератор наш набор данных и поддерживает автоматическое разделение на группы, сэмплирование, перемешивание и мультипроцессную загрузку данных. Здесь мы определяем группу (`batch`) размером 64, т.е. каждый элемент загрузчика данных итерирует по группе из 64-х элементов данных и лейблов.

In [3]:
batch_size = 64

# Создать загрузчика данных.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for x, y in test_dataloader:
    print(f"Shape of x [n, c, h, w]: {x.shape}")
    print(f"Shape of y: {y.shape}")


Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Shape of x [n, c

## Создание модели

Для объявления нейросети в PyTorch мы создаём класс, который наследуется от `nn.Module`. Мы объявляем слои нейросети в методе `__init__` и определяем как данные будут проходить через нейросеть в методе `forward`. Для ускорения операций в нейросети, мы используем GPU или MPS, если доступны.

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Почитать больше о [построении нейросетей в PyTorch](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

# Оптимизация параметров модели

Для тренировки модели нам необходимы [функция потерь](https://pytorch.org/docs/stable/nn.html#loss-functions) и [оптимизатор](https://pytorch.org/docs/stable/optim.html).

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


В одном цикле обучения модель делает предсказания на тренировочном наборе данных (подаваемого в неё группами) и распространяет обратно ошибку для улучшения параметров.

In [6]:
from tqdm import tqdm

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # Общее количество данных
    model.train()

    # Создаем прогресс-бар
    progress_bar = tqdm(total=size, desc="Training", unit="sample")

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        # Вычисление ошибки предсказания
        pred = model(x)
        loss = loss_fn(pred, y)

        # Обратное распространение ошибки
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Обновляем прогресс-бар на количество обработанных элементов
        progress_bar.update(len(x))

        # Логгируем каждые 100 батчей
        if batch % 100 == 0:
            loss_value = loss.item()
            current = batch * len(x)
            # print(f"Loss: {loss_value:.6f} [{current}/{size}]")

    progress_bar.close()


Также мы проверяем производительность модели на тестовом наборе данных, чтобы убедиться, что она обучается.

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


Процесс обучения проводится через несколько итерация (эпох). В течение каждой эпохи модель обучает параметры делать лучшие предсказания. Вы выводим точность модели и потери на каждой эпохе. Хотелось бы видеть увеличение точности и уменьшение потерь каждую эпоху.

In [8]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------------------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(train_dataloader, model, loss_fn)

print("Done!")


Epoch 1
--------------------------------------------------------------------


Training: 100%|██████████| 60000/60000 [00:25<00:00, 2391.74sample/s]


Test Error: 
 Accuracy: 51.1%, Avg loss: 2.159808 

Epoch 2
--------------------------------------------------------------------


Training: 100%|██████████| 60000/60000 [00:15<00:00, 3976.93sample/s]


Test Error: 
 Accuracy: 61.1%, Avg loss: 1.896836 

Epoch 3
--------------------------------------------------------------------


...

Test Error: 
 Accuracy: 64.2%, Avg loss: 1.524458 

Epoch 4
--------------------------------------------------------------------


Training: 100%|██████████| 60000/60000 [00:15<00:00, 3844.99sample/s]


Test Error: 
 Accuracy: 64.6%, Avg loss: 1.251977 

Epoch 5
--------------------------------------------------------------------


Training: 100%|██████████| 60000/60000 [00:14<00:00, 4009.82sample/s]


Test Error: 
 Accuracy: 65.7%, Avg loss: 1.081538 

Done!


## Сохранение модели

Общий подход сохранения модели это сериализация внутреннего состояния словаря (содержащего параметры модели).

In [9]:
MODEL_FILE: str = "model.pth"

state_dict = model.state_dict()

torch.save(state_dict, MODEL_FILE)
print(f"Saved PyTorch Model State to {MODEL_FILE}")


Saved PyTorch Model State to model.pth


## Загрузка модели

Процесс загрузки модели включает в себя пересоздание структуры модели и загрузки состояния словаря в неё.

In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(MODEL_FILE, weights_only=True))

print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Теперь эта модель может использоваться для предсказаний.

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

data, label = 0, 1

for idx in range(len(classes)):
    x, y = test_data[idx][data], test_data[idx][label]

    with torch.no_grad():
        x = x.to(device)
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f"Predicted: \"{predicted}\", Actual: \"{actual}\", result: \"{predicted == actual}\"")


Predicted: "Ankle boot", Actual: "Ankle boot", result: "True"
Predicted: "Pullover", Actual: "Pullover", result: "True"
Predicted: "Trouser", Actual: "Trouser", result: "True"
Predicted: "Trouser", Actual: "Trouser", result: "True"
Predicted: "Pullover", Actual: "Shirt", result: "False"
Predicted: "Trouser", Actual: "Trouser", result: "True"
Predicted: "Coat", Actual: "Coat", result: "True"
Predicted: "Pullover", Actual: "Shirt", result: "False"
Predicted: "Sneaker", Actual: "Sandal", result: "False"
Predicted: "Sneaker", Actual: "Sneaker", result: "True"
